## Populate Task

In [ ]:
import sys
from pathlib import Path

# Đường dẫn tới thư mục gốc của dự án
project_root = Path('C:/Users/ADMIN/Desktop/DATN/Extract_Information')
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

In [ ]:
task = 'open_three_shots'
model_name = 'Llama-3'

In [11]:
PATH_FILE_TASK = 'C:/Users/ADMIN/Desktop/DATN/Extract_Information/tasks/task_template/'
PATH_FILE_TEMPLATE = 'C:/Users/ADMIN/Desktop/DATN/Extract_Information/tasks/tasks/task_template.json'
PATH_FILE_DATA = 'C:/Users/ADMIN/Desktop/DATN/Extract_Information/data/mave_filtered_test.jsonl'

if model_name == 'GPT':
    PATH_FILE_RESULT = 'C:/Users/ADMIN/Desktop/DATN/Extract_Information/tasks/result/GPT'
else:
    PATH_FILE_RESULT = 'C:/Users/ADMIN/Desktop/DATN/Extract_Information/tasks/result/Llama'

In [13]:
from dotenv import load_dotenv

# Load OPENAI_API_KEY from .env file
load_dotenv()

True

In [14]:
import json
from pieutils.pieutils import update_task_dict_from_file, combine_example, calculate_recall_precision_f1, parse_gpt_response_to_dict

# Load task template
with open(PATH_FILE_TEMPLATE, 'r') as f:
    task_dict = json.load(f)

task_dict['task_prefix'] = "Extract all important attribute name : attribute value pairs " \
                            "mentioned in the product title below and return the attribute name : attribute value pairs. " \
                            "All parts of the product title should be assigned to an attribute. \n " \
                            "Desired format: <attribute name>: <attribute value> \n <attribute name>: <attribute value>"
task_dict['shots'] = 3

# Generate examples
task_dict = update_task_dict_from_file(PATH_FILE_DATA, task_dict, False)

with open(PATH_FILE_TASK + 'task_{}.json'.format(task), 'w', encoding='utf-8') as f:
    json.dump(task_dict, f, indent=4)

## Evaluate Task

In [ ]:
# Initialize processing
from tqdm import tqdm
import torch
import transformers
from langchain import LLMChain, HuggingFacePipeline
from langchain.chat_models import ChatOpenAI

from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate
)
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.callbacks import get_openai_callback

In [ ]:
# Model
if model_name == 'GPT':
    llm = ChatOpenAI(model_name='gpt-3.5-turbo-0125', temperature=0)
else:
    tokenizer = AutoTokenizer.from_pretrained('/ceph/alebrink/cache/models--meta-llama--Meta-Llama-3-8B/snapshots/62bd457b6fe961a42a631306577e622c83876cb6')
    model = AutoModelForCausalLM.from_pretrained(
        '/ceph/alebrink/cache/models--meta-llama--Meta-Llama-3-8B/snapshots/62bd457b6fe961a42a631306577e622c83876cb6',
        torch_dtype=torch.float32,
        device_map="auto",
        offload_folder="save_folder",
    )
    model.tie_weights()

    hf_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        device_map="auto"
    )

    terminators = [
        hf_pipeline.tokenizer.eos_token_id,
        hf_pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    llm = HuggingFacePipeline(pipeline=hf_pipeline)

In [15]:
# Put task prefix into system message.
prompt_list = []
system_message_prompt = SystemMessagePromptTemplate.from_template(task_dict['task_prefix'])
prompt_list.append(system_message_prompt)

# Prepare three-shot  
human_example_template="{human_msg_0}"
human_example_message_prompt = HumanMessagePromptTemplate.from_template(human_example_template)
prompt_list.append(human_example_message_prompt)
ai_example_template="{ai_msg_0}"
ai_example_message_prompt = AIMessagePromptTemplate.from_template(ai_example_template)
prompt_list.append(ai_example_message_prompt)

human_example_template="{human_msg_1}"
human_example_message_prompt = HumanMessagePromptTemplate.from_template(human_example_template)
prompt_list.append(human_example_message_prompt)
ai_example_template="{ai_msg_1}"
ai_example_message_prompt = AIMessagePromptTemplate.from_template(ai_example_template)
prompt_list.append(ai_example_message_prompt)

human_example_template="{human_msg_2}"
human_example_message_prompt = HumanMessagePromptTemplate.from_template(human_example_template)
prompt_list.append(human_example_message_prompt)
ai_example_template="{ai_msg_2}"
ai_example_message_prompt = AIMessagePromptTemplate.from_template(ai_example_template)
prompt_list.append(ai_example_message_prompt)

prompt_list.append(system_message_prompt)

# Add 
human_template="{input_string}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
prompt_list.append(human_message_prompt)

chat_prompt = ChatPromptTemplate.from_messages(prompt_list)


llm_chain = LLMChain(
    prompt=chat_prompt,
    llm=llm
)

# Save model
task_dict['model'] = model_name

In [16]:
def select_and_run_llm(human0, system0, human1, system1, human2, system2, input):
        pred = None
        if len(input) == 0:
            # No input text provided.
            return pred
        if model_name == 'GPT':
            response = llm_chain.run({'human_msg_0': human0, 'ai_msg_0': system0,'human_msg_1': human1, 'ai_msg_1': system1,'human_msg_2': human2, 'ai_msg_2': system2,'input_string': input})
        else:
            messages = [{"role": "system", "content": chat_prompt.messages[0].format().content},
                        {"role": "system", "content": chat_prompt.messages[1].format(human_msg_0 = human0).content},
                        {"role": "system", "content": chat_prompt.messages[2].format(ai_msg_0 = system0).content},
                        {"role": "system", "content": chat_prompt.messages[3].format(human_msg_1 = human1).content},
                        {"role": "system", "content": chat_prompt.messages[4].format(ai_msg_1 = system1).content},
                        {"role": "system", "content": chat_prompt.messages[5].format(human_msg_2 = human2).content},
                        {"role": "system", "content": chat_prompt.messages[6].format(ai_msg_2 = system2).content},
                        {"role": "system", "content": chat_prompt.messages[7].format().content},
                        {"role": "human", "content": chat_prompt.messages[8].format(input_string = input).content}]

            hf_prompt = hf_pipeline.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )

            hf_outputs = hf_pipeline(hf_prompt, max_new_tokens=256,
                                    eos_token_id=terminators,
                                    do_sample=True,
                                    temperature=0,
                                    top_p=0.9
                                    )
            response = hf_outputs[0]["generated_text"][len(hf_prompt):] 
        
        try:
            pred = parse_gpt_response_to_dict(response)
        except Exception as e:
            print(e)
            print('Response: ')
            print(response)
        return pred

In [17]:
# General task prefix
from tqdm import tqdm
from langchain.callbacks import get_openai_callback

if model_name == 'GPT':
    with get_openai_callback() as cb:
        preds = [select_and_run_llm(example['human_0'], example['system_0'], example['human_1'], example['system_1'], example['human_2'], example['system_2'], example['input']) for example in tqdm(task_dict['examples'])]
        
        task_dict['total_tokens'] = cb.total_tokens
        print(f"Total Tokens: {cb.total_tokens}")
else:
    preds = [select_and_run_llm(example['human_0'], example['system_0'], example['human_1'], example['system_1'], example['human_2'], example['system_2'], example['input']) for example in tqdm(task_dict['examples'])]

100%|██████████| 562/562 [12:08<00:00,  1.30s/it]

Total Tokens: 273264


In [18]:
print(preds)

[{'Capacity': '64GB', 'Brand': 'Kyocera Hydro VIBE', 'SD Format': 'MicroSDHC', 'Speed': 'High speed, lossless recording', 'Class': 'Class 10', 'Speed Rating': '38MB/sec', 'Included Items': 'Standard SD Adapter'}, {'Brand': 'Kingston', 'Capacity': '16GB', 'SD Format': 'MicroSDHC', 'Compatibility': 'LG G Stylo', 'Custom Formatting': 'Yes', 'Included Items': 'Standard SD Adapter', 'Speed': '32Mbps / Class 4'}, {'Brand': 'Essential ULTRA', 'Capacity': '32GB', 'Retailer': 'Best Buy', 'Compatibility': 'Insignia Flex', 'SD Format': 'MicroSDHC', 'Custom Format': 'Hi-Speed Lossless certified recording', 'Included Items': 'SD Adapter', 'Class': 'Class 10', 'Speed': 'Up to 500x or 70MB/sec'}, {'Brand': 'SanDisk', 'Capacity': '32GB', 'Compatibility': 'Nokia Lumia 521 Smartphone', 'SD Format': 'MicroSDHC', 'Speed': 'High speed, lossless recording', 'Class': 'UHS-1 Class 10'}, {'Brand': 'Kingston', 'Capacity': '4GB', 'SD Format': 'MicroSDHC', 'Compatibility': 'Nokia Lumia 520 Smartphone', 'Custom Fo

In [19]:
# # Load task template
# with open(PATH_FILE_RESULT + 'task_run_chat_open_extraction_gpt-3.5-turbo-0125.json', 'r') as f:
#     task_dict = json.load(f)

targets = [example['target_scores'] for example in task_dict['examples']]
categories = [example['category'] for example in task_dict['examples']]
attributes = [example['attribute'] for example in task_dict['examples']]

postprocessed_preds = [pred[attribute] if attribute in pred else "I do not know." for pred, attribute in zip(preds, attributes)]
#postprocessed_preds = [example['post_pred'] for example in task_dict['examples']]

task_dict['examples'] = [combine_example(example, pred, post_pred) 
                     for example, pred, post_pred in zip(task_dict['examples'], preds, postprocessed_preds)]

results = calculate_recall_precision_f1(targets, postprocessed_preds, categories, attributes)

Attribute: SD Format - Category: Flash Memory Cards
{'precision': 92.11, 'recall': 85.37, 'f1': 88.61}
Attribute: Capacity - Category: Flash Memory Cards
{'precision': 100.0, 'recall': 92.5, 'f1': 96.1}
Attribute: Optical Zoom - Category: Digital Cameras
{'precision': 100.0, 'recall': 100.0, 'f1': 100.0}
Attribute: Sensor Size - Category: Digital Cameras
{'precision': 100.0, 'recall': 7.14, 'f1': 13.33}
Attribute: Sensor Type - Category: Digital Cameras
{'precision': 89.47, 'recall': 85.0, 'f1': 87.18}
Attribute: Resolution - Category: Digital Cameras
{'precision': 97.56, 'recall': 97.56, 'f1': 97.56}
Attribute: Processor Speed - Category: Laptops
{'precision': 97.3, 'recall': 81.82, 'f1': 88.89}
Attribute: Battery Life - Category: Laptops
{'precision': 100.0, 'recall': 100.0, 'f1': 100.0}
Attribute: Screen Size - Category: Laptops
{'precision': 100.0, 'recall': 100.0, 'f1': 100.0}
Attribute: Number of Cores - Category: Laptops
{'precision': 77.78, 'recall': 17.5, 'f1': 28.57}
Attribut

In [20]:
# Save populated task
result_file = PATH_FILE_RESULT + '{}_{}.json'.format(task, task_dict['model'])
with open(result_file, 'w') as fp:
    json.dump(task_dict, fp, indent=4)

In [21]:
# Error Analysis
print('Prompts for which target and postprocessed prediction do not match.')
print('-----------')
input_texts = [example['input'] for example in task_dict['examples']]
preds = [example['pred'] for example in task_dict['examples']]

for input_text, target, pred, post_pred in zip(input_texts, targets, preds, postprocessed_preds):
    if post_pred not in target.keys():
        #print('Prompt: {}'.format(chat_prompt.format(task_prefix=task_dict['task_prefix'], input_string= input_text, human_msg_0= human_text_0, ai_msg=system_text,)))
        print('Input: {}'.format(input_text))
        print('Prediction: \n {}'.format(pred))
        print('Prediction 2: {}'.format(post_pred))
        print('Target: {}'.format(target))
        print('-----------')

Prompts for which target and postprocessed prediction do not match.
-----------
Input: Thinkpad Z60T 2511 Pentium M 740 1.73 Ghz-ram 512 Mb-hd 80 Gb-DVD-writer-cellula
Prediction: 
 {'Brand': 'Thinkpad', 'Model': 'Z60T 2511', 'Processor': 'Pentium M 740 1.73 GHz', 'RAM': '512 MB', 'Hard Drive Capacity': '80 GB', 'Optical Drive': 'DVD writer', 'Cellular Connectivity': 'Yes'}
Prediction 2: I do not know.
Target: {'Pentium M': 1, 'Intel Pentium M': 1, 'Intel': 1}
-----------
Input: Apple MacBook Pro MA601LL/A 15.4" Laptop (2.16 GHz Intel Core Duo, 1 GB RAM, 100 GB Hard Drive, SuperDrive)
Prediction: 
 {'Brand': 'Apple', 'Model': 'MacBook Pro MA601LL/A', 'Screen Size': '15.4', 'Processor': '2.16 GHz Intel Core Duo', 'RAM': '1 GB', 'Hard Drive Capacity': '100 GB', 'Optical Drive': 'SuperDrive'}
Prediction 2: I do not know.
Target: {'Intel Core Duo': 1, 'Intel': 1}
-----------
Input: Sony DSC-H5S Cyber-Shot 7.2 Megapixel Digital Camera (Silver)
Prediction: 
 {'Brand': 'Sony', 'Model': 'DSC-H